# kube-prometheus (prometheus + grafana + dashboard)

https://github.com/prometheus-operator/kube-prometheus

Quickstart: https://github.com/prometheus-operator/kube-prometheus#quickstart

## Installing prometheus operator

In [ ]:
! cd /tmp && git clone https://github.com/prometheus-operator/kube-prometheus.git

In [ ]:
! cd /tmp/kube-prometheus && kubectl apply --server-side -f manifests/setup
! until kubectl get servicemonitors --all-namespaces ; do date; sleep 1; echo ""; done

In [ ]:
! cd /tmp/kube-prometheus && kubectl apply -f manifests/

In [ ]:
! kubectl get all -n monitoring

In [1]:
# clean up
# kubectl delete --ignore-not-found=true -f manifests/ -f manifests/setup

## Adding an app to be discovered by prometheus

[prometheus operator example](https://github.com/prometheus-operator/kube-prometheus/tree/main/examples/example-app)

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/prometheus-operator/kube-prometheus/main/examples/example-app/example-app.yaml

In [30]:
# Get endpoints of the example app
! kubectl describe svc example-app | grep Endpoints

Endpoints:         10.233.92.7:8080,10.233.92.8:8080,10.233.96.8:8080 + 1 more...


In [62]:
# The examples app exposes /metrics :
! ssh -q node1 "curl http://10.233.92.7:8080/metrics" | head

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0# HELP demo_api_http_requests_in_progress The current number of API HTTP requests in progress.
# TYPE demo_api_http_requests_in_progress gauge
demo_api_http_requests_in_progress 0
# HELP demo_api_request_duration_seconds A histogram of the API HTTP request durations in seconds.
# TYPE demo_api_request_duration_seconds histogram
demo_api_request_duration_seconds_bucket{method="GET",path="/api/bar",status="200",le="0.0001"} 0
demo_api_request_duration_seconds_bucket{method="GET",path="/api/bar",status="200",le="0.00015000000000000001"} 0
demo_api_request_duration_seconds_bucket{method="GET",path="/api/bar",status="200",le="0.00022500000000000002"} 0
demo_api_request_duration_seconds_bucket{method="GET",path="/api/bar",status="200",le="0.0003375"} 0
demo_ap

In [ ]:
# http://127.0.0.1:9090/api/v1/targets
! kubectl describe svc prometheus-k8s -n monitoring

In [69]:
# The prometheus-k8s-* pods expose /metrics with all metrics of all targets combined
# However, the example apps metrics are NOT included
! ssh -q node3 curl -s  http://10.233.92.6:9090/metrics | grep demo_api_request_duration_seconds_bucket

### How to I make prometheus discover the example app?

[kubernetes_sd_config in the prometheus docs](https://prometheus.io/docs/prometheus/latest/configuration/configuration/#kubernetes_sd_config)